In [ ]:
import pandas as pd
import pymc3 as pm
import arviz as az
import matplotlib.pyplot as plt
import seaborn as sns
import theano.tensor as tt

In [ ]:
i = 2
print(all_teams[i])
print(str(number_team[i]))

In [ ]:
# pd.read_html("https://www.naqt.com/stats/tournament/standings.jsp?tournament_id=9500")[1]

In [ ]:
result.to_csv("data/03_primary/all_games.csv", index=False)

# Analyze game data

In [ ]:
# Process all games
all_games = io.load("2018_all_games")
rename_dict = {
    "P":"team_1_powers",
    "TU":"team_1_tens",
    "I":"team_1_negs",
    "B":"team_1_bonus_points",
    "PPB":"team_1_ppb",
    "P.1":"team_2_powers",
    "TU.1":"team_2_tens",
    "I.1":"team_2_negs",
    "B.1":"team_2_bonus_points",
    "PPB.1":"team_2_ppb",
    "team":"team_1",
    "Opponent":"team_2"
}
print(all_games.shape)
all_games = (all_games
             .rename(rename_dict, axis=1)
             .query("Score != 'Forfeit'")
             .eval("team_1_score = team_1_powers * 15 + team_1_tens * 10 - team_1_negs * 5 + team_1_bonus_points")
             .eval("team_2_score = team_2_powers * 15 + team_2_tens * 10 - team_2_negs * 5 + team_2_bonus_points")
             .eval("point_diff = team_1_score - team_2_score")
             .drop(["Result"], axis=1)
            )
all_games["point_diff_normalized"] = (all_games["point_diff"] - all_games["point_diff"].mean()) / all_games["point_diff"].std()

# Add a unique numeric index for each team
team_indices = (pd.DataFrame(list(set(all_games["team_1"].unique()) | set(all_games["team_2"].unique())))
                .reset_index()
                .rename({0:"team"}, axis=1)
                .set_index("team")
                .to_dict()["index"]
               )
all_games["team_1_index"] = all_games["team_1"].map(team_indices)
all_games["team_2_index"] = all_games["team_2"].map(team_indices)

assert all_games[all_games["team_1_index"].isna()].empty
assert all_games[all_games["team_2_index"].isna()].empty

# Find and remove duplicate games
all_games["teams"] = [tuple(sorted(x)) for x in zip(all_games["team_1"], all_games["team_2"])]
all_games = (all_games
             .drop_duplicates(["Round", "teams"])
             .drop(["teams"], axis=1)
            )

print(all_games.shape)
all_games.columns = all_games.columns.str.lower()

In [ ]:
all_games.head()

### Model

Model and code are based on [this article](http://danielweitzenfeld.github.io/passtheroc/blog/2014/10/28/bayes-premier-league/).

Our goal is to come up with an underlying team strength parameter as well as uncertainty around that team strength parameter. Let $y_{gj}$ be the observed score for team $j$ in game $g$. We model the score using a Poisson distribution i.e. $y_{gj}|\theta_{gj} \sim Poisson(\theta_{gj})$. Note that there is one $\theta_{gj}$ for each team in each round. 

At the next level of the model, we model each $\theta$ as a log-linear function:
$$
\log \theta_{g1} = attack_1 - defense_2 \\
\log \theta_{g2} = attack_2 - defense_1
$$
i.e. we assume there is an attack and defense strength for each team. These parameters are modeled as a hierarchical model where, for each team $t$,
$$
attack_t \sim N(\mu_{attack},\tau_{attack}) \\
defense_t \sim N(\mu_{defense},\tau_{defense})
$$
And in turn we have hyperpriors where $\mu_{attack}, \mu_{defense} \sim N(.,.)$ and $\tau_{attack}, \tau_{defense} \sim Gamma(.,.)$

To ensure identifiability, we make the attack and defense parameters sum to 0:
$$
\sum_{t \in Teams} attack_t = 0 \\
\sum_{t \in Teams} defense_t = 0
$$


In [ ]:
num_teams = len(set(all_games["team_1"].unique()) | set(all_games["team_2"].unique()))
team_1 = all_games["team_1_index"].values
team_2 = all_games["team_2_index"].values

In [ ]:
# %debug
with pm.Model() as model:
    
    #hyperpriors
    tau_attack = pm.Gamma('tau_attack', alpha=20, beta=20)
    tau_defense = pm.Gamma('tau_defense', alpha=20, beta=20)
#     intercept = pm.HalfNormal('intercept', sd=1)

    # team-specific model parameters
    atts_star = pm.Normal("atts_star", mu=0, sd=tau_attack, shape=num_teams)
    defs_star = pm.Normal("defs_star", mu=0, sd=tau_defense, shape=num_teams)
    
    atts = pm.Deterministic('atts', atts_star - tt.mean(atts_star))
    defs = pm.Deterministic('defs', defs_star - tt.mean(defs_star))
    team_1_theta = tt.exp(atts[team_1] + defs[team_2])
    team_2_theta = tt.exp(atts[team_2] + defs[team_1])
    
    # likelihood of observed data
    team_1_points = pm.Poisson('team_1_points', mu=team_1_theta, observed=all_games["team_1_score"])
#     team_2_points = pm.Poisson('team_2_points', mu=team_2_theta, observed=all_games["team_2_score"])

    prior = pm.sample_prior_predictive(samples=1000)
    trace = pm.sample()
    posterior_predictive = pm.sample_posterior_predictive(trace)

In [ ]:
pm.traceplot(trace)

In [ ]:
team_ranks = (trace.get_values("atts")).mean(axis=0).argsort()[::-1]
team_indices_reverse = {val:key for key, val in team_indices.items()}
[team_indices_reverse[idx] for idx in team_ranks]

In [ ]:
data = az.from_pymc3(
        trace=trace,
        prior=prior,
        posterior_predictive=posterior_predictive,
    )

In [ ]:
model.check_test_point()

In [ ]:
prior.keys()

In [ ]:
print(prior["team_1_points"].mean(), prior["team_1_points"].max())
# print(prior["team_2_points"].mean(), prior["team_2_points"].max())

In [ ]:
sns.distplot(prior["team_1_points"].flatten())

In [ ]:
# %debug


with pm.Model() as model:

    #hyperpriors
    sd_team_strength = pm.Gamma("sd_team_strength", alpha=1, beta=1)
    sd_likelihood = pm.Gamma("sd_likelihood", alpha=1, beta=1)

    # team-specific model parameters
    team_strength = pm.Normal("team_strength", mu=0, sd=sd_team_strength, shape=num_teams)
    strengths = pm.Deterministic('strengths', team_strength - tt.mean(team_strength))

    # likelihood of observed data
    mu = strengths[team_1] - strengths[team_2]
    point_diff = pm.Normal('point_diff', mu=mu, sd=sd_likelihood, observed=all_games["point_diff"])
    
    prior = pm.sample_prior_predictive(samples=1000)
    trace = pm.sample()
    posterior_predictive = pm.sample_posterior_predictive(trace)

In [ ]:
pm.traceplot(trace)

In [ ]:
# from simulation_based_calibration import SBC, plot_sbc
# sbc = SBC(my_model, "y",
#         num_simulations=10)

# sbc.run_simulations()

In [ ]:
prior.keys()

In [ ]:
prior["point_diff"].min(), prior["point_diff"].mean(), prior["point_diff"].max()

In [ ]:
sns.distplot(prior["team_strength"].flatten(), kde=False)